In [ ]:
import pandas as pd
import numpy as np
from scapy.all import *
import dpkt

# Chunk Detection

In [1]:

from math import *
import scapy.layers as layers
IP = layers.inet.IP

GET_THRESH = 300 # bytes
DOWN_THRESH = 300  # bytes
VIDEO_CHUNK_GETSIZE = 700 # bytes
AUDIO_CHUNK_GETSIZE = 600 # bytes
NETINFO_NUM = 25

class Chunk():
    def __init__(self, start_time = 0, server_ip='',ttfb = 0, download_time = 0, slack_time = 0, get_size=0, chunk_size = 0, type="", protocol=""):
        self.start_time = start_time
        self.server_ip = server_ip
        self.ttfb = ttfb
        self.download_time = download_time
        self.slack_time = slack_time
        self.get_size = get_size
        self.chunk_size = chunk_size
        self.type = type
        self.protocol = protocol
    def __str__(self):
        return f"{self.start_time} {self.ttfb} {self.download_time} {self.slack_time} {self.get_size} {self.chunk_size} {self.type} {self.protocol}"
    def __repr__(self):
        return f"{self.start_time} {self.ttfb} {self.download_time} {self.slack_time} {self.get_size} {self.chunk_size} {self.type} {self.protocol}"


def isUplink(p):
    # IP = scapy.layers.inet.IP
    return p[IP].src.startswith('192.168.')

def detectAV(c):
    if abs(c.get_size-VIDEO_CHUNK_GETSIZE) > abs(c.get_size-AUDIO_CHUNK_GETSIZE):
        flag=0
    else:
        flag=1
    if c.chunk_size<=80*1024:
        flag=2
    return flag

def chunkDetection(filename):
    a = rdpcap(filename)
    meta_time = float(a[0].time)
    chunk = {}
    chunks = []
    downFlag = {}
    # IP = scapy.layers.inet.IP
    for p in a:
        if p.haslayer(IP):
            ipSrc=p[IP].src
            ipDst=p[IP].dst
            pLen=p[IP].len
            pHdr=p[IP].ihl*4
            ip_time=float(p.time)
            if isUplink(p) and pLen-pHdr > GET_THRESH:
                if ipDst in chunk:
                    chunk[ipDst].slack_time = ip_time - chunk[ipDst].download_time
                    avFlag=detectAV(chunk[ipDst])
                    if avFlag==0:
                        # chunk[ipDst].type='a'
                        chunks.append(chunk[ipDst])
                    elif avFlag==1:
                        # chunk[ipDst].type='v'
                        chunks.append(chunk[ipDst])
                    else:
                        chunk.pop(ipDst)
                        downFlag.pop(ipDst)
                chunk[ipDst] = Chunk(start_time=ip_time, get_size=pLen-pHdr, server_ip=ipDst)
                downFlag[ipDst] = False
            elif not isUplink(p) and pLen > DOWN_THRESH:
                if ipSrc in chunk:
                    if not downFlag[ipSrc]:
                        chunk[ipSrc].ttfb = ip_time
                        downFlag[ipSrc] = True
                    chunk[ipSrc].download_time = ip_time
                    chunk[ipSrc].chunk_size += pLen - pHdr
                    chunk[ipSrc].protocol = p.proto
    
    for c in chunk.values():
        avFlag=detectAV(c)
        if avFlag==0:
            # c.type='a'
            chunks.append(c)
        elif avFlag==1:
            # c.type='v'
            chunks.append(c)
    return meta_time, chunks,chunk

filename = 'RequetDataSetNew/A0/PCAP_FILES/baseline_Jan17_exp_31.pcap'
meta_time, chunks, chunk=chunkDetection(filename)

AttributeError: module 'scapy.layers' has no attribute 'inet'

In [ ]:
print(len(chunks))
print(len(chunk))
print(len(chunk.values()))

# 阮阮Test

In [2]:
import dpkt
import numpy as np
import socket
np.set_printoptions(suppress=True,   precision=20,  threshold=10,  linewidth=40)
GET_THRESH = 300  # bytes
DOWN_THRESH = 300  # bytes
VIDEO_CHUNK_GETSIZE = 700  # bytes
AUDIO_CHUNK_GETSIZE = 600  # bytes

class Chunk():
    def __init__(self, GetTimestamp=0, GetSize=0, DownStart=0, DownEnd=0, DownSize=0, type=0, GetProtocol="", serverIP = ""):
        self.GetTimestamp = GetTimestamp  #发送请求的时间戳
        self.GetSize = GetSize  #Get请求的长度
        self.DownStart = DownStart  #块的第一个下行包的时间戳
        self.DownEnd = DownEnd  #块的最后一个下行包的时间戳
        self.DownSize = DownSize  #块的所有下行包的大小之和
        self.type = type  #类型：视频或音频
        self.GetProtocol = GetProtocol  #协议
        self.serverIP = serverIP
    def getGetTimestamp(self):
        return self.GetTimestamp

    def getDownEnd(self):
        return  self.DownEnd

    def getServerIP(self):
        return self.serverIP

    def getGetSize(self):
        return self.GetSize

    def setType(self, newType):
        self.type = newType

    def __lt__(self, other):
        self_small = True
        A = self.GetTimestamp
        B = other.GetTimestamp
        self_small = A < B
        return self_small

    def detectAV(self):
        # 论文里是根据GetSize, DownSize, GetProtocol来区分视频块，音频块和后台流量的
        flag = 0
        if self.DownSize <= 80*1024:
            flag = 2
        else:
            if abs(self.GetSize-VIDEO_CHUNK_GETSIZE) > abs(self.GetSize-AUDIO_CHUNK_GETSIZE): #判断这个大小更接近视频块还是音频块
                flag = 0
            else:
                flag = 1
        # 基于协议号的筛选
        return flag

    def __str__(self):
        return f"{self.serverIP} {self.GetTimestamp} {self.GetSize} {self.DownStart} {self.DownEnd} {self.DownSize} {self.type} {self.GetProtocol}"

    def __repr__(self):
        return f"{self.serverIP} {self.GetTimestamp} {self.GetSize} {self.DownStart} {self.DownEnd} {self.DownSize} {self.type} {self.GetProtocol}"

def inet_to_str(inet):
    try:
        return socket.inet_ntop(socket.AF_INET, inet)
    except:
        return False

def isUplink(src):
    return src[0:7] == '192.168'


def ChunkDetection(filename):
    #解析pcap文件
    print("Reading pcap file...")

    #读入pcap文件
    f = open(filename, 'rb')
    pcap = dpkt.pcap.Reader(f)

    # meta_time = 0
    ii = 0
    #pcap是Reader类，无法切片，目前先这样写
    # for ts, buf in pcap:
    #     ii += 1
    #     if ii == 1:
    #         meta_time = ts
    #         break
    # end_time = meta_time
    chunks = {}
    chunksValue = []
    downFlag = {}  #用于标识是不是第一个下行报文

    print("Parsing pcap file...")

    times = []
    ip_srcs = []
    ip_dsts = []
    ip_lens = []
    ip_ihls = []
    ip_protos = []
    for ts, buf in pcap:
        # 这里也是对没有IP段的包过滤掉
        eth = dpkt.ethernet.Ethernet(buf)
        if eth.type != dpkt.ethernet.ETH_TYPE_IP:
            continue
        # ts是时间戳
        times.append(ts)
        ip = eth.data
        ip_src = inet_to_str(ip.src)  #
        ip_dst = inet_to_str(ip.dst)  #
        ip_srcs.append(ip_src)  # src
        ip_dsts.append(ip_dst)  # dst
        ip_lens.append(ip.len)  # len
        ip_protos.append(ip.p)  # proto:协议号tcp是6
        ip_ihls.append(ip.hl)  # ihl
    first_end_time = (times[0],times[-1])
    print("Calcilate chunk...")
    sumGetSize = 0
    for n in range(len(ip_srcs)):
        ipSrc = ip_srcs[n]
        ipDst = ip_dsts[n]
        ipSize = ip_lens[n] - ip_ihls[n] * 4
        ipTime = times[n]
        ipProto = ip_protos[n]
        #end_time = ipTime
        if isUplink(ipSrc) and ipSize > GET_THRESH:
            # 上行包，2种情况：（1）已经出现过的，则处理上一个发往这个站点的块，并初始化新块；（2）没有出现过，初始化新块
            if ipDst in chunks:  #意味着以这一ip地址为目的地址的块已经结束，以这一ip地址为目的地址的块马上开始
                #筛选：刚刚结束的块是否是音频或视频块
                avFlag = chunks[ipDst].detectAV()
                if not avFlag == 2:  # 音频块
                    sumGetSize += chunks[ipDst].getGetSize()
                    chunksValue.append(chunks[ipDst])
                else:  # 后台流量
                    chunks.pop(ipDst)  # 抛弃这个块
                    downFlag.pop(ipDst)
            #初始化新块
            chunks[ipDst] = Chunk(GetTimestamp=ipTime, GetSize=ipSize, GetProtocol=ipProto, serverIP = ipDst)
            downFlag[ipDst] = False
        elif not isUplink(ipSrc) and ipSize > DOWN_THRESH:
            if ipSrc in chunks:
            # 下行包，2种情况：（1）是Get请求的第一个下行包，记录时间，更新大小和时间；（2）不是Get请求的第一个下行包，更新大小和时间
                if not downFlag[ipSrc]:
                    chunks[ipSrc].DownStart = ipTime  # 收到第一个下行包的时间
                    downFlag[ipSrc] = True
                chunks[ipSrc].DownEnd = ipTime
                chunks[ipSrc].DownSize += ipSize
                chunks[ipSrc].protocol = ipProto

    for c in chunks.values():
        avFlag = c.detectAV()
        if not avFlag == 2:  # 音频块
            sumGetSize += c.getGetSize()
            chunksValue.append(c)

    # 区分音频块和视频块
    chunkNum = len(chunksValue)
    ave_GetSize = sumGetSize / chunkNum
    for s_chunk in chunksValue:
        if (s_chunk.getGetSize() > ave_GetSize):
            s_chunk.setType(0)
        else:
            s_chunk.setType(1)

    return chunkNum, chunksValue,first_end_time

def getChunkMetrics(chunkNum, chunksValue):
    sortChunks = sorted(chunksValue)
    #顺序：'start_time', 'type', 'ttfb', 'download_time', 'end_time', 'get_size', 'chunk_size'
    output_data = np.zeros((chunkNum, 7))
    i = 0
    for c in sortChunks:
        start_time = c.GetTimestamp
        ttfb = c.DownStart - start_time
        download_time = c.DownEnd - c.DownStart
        end_time = c.DownEnd
        get_size = c.GetSize
        chunk_size = c.DownSize
        type = c.type
        output_data[i] = np.array([start_time, type, ttfb, download_time, end_time, get_size, chunk_size])
        i += 1
    print(i)
    return output_data

filename = 'RequetDataSetNew/A0/PCAP_FILES/baseline_Jan17_exp_31.pcap'
# 1. chunkDetection
chunkNum, chunksValue0,fe_time = ChunkDetection(filename)
# 2. chunkMetrics
output = getChunkMetrics(chunkNum, chunksValue0)

Reading pcap file...
Parsing pcap file...
Calcilate chunk...
56


In [5]:
def getFeature(epoch_msec,cm,first_end_time):
    t = epoch_msec / 1000
    if t < first_end_time[0] or t > first_end_time[1]:
        return None
    s = []
    start_time = 0
    type = 1
    ttfb = 2
    download_time = 3
    end_time = 4
    get_size = 5
    chunk_size = 6
    flag = True
    last_chunk=[]
    for w in range(1, 21):
        period = w * 10.0
        if t - period < first_end_time[0]:
            total_number_of_chunks_v = -1
            avg_chunk_size_v = -1
            download_time_v = -1
            total_number_of_chunks_a = -1
            avg_chunk_size_a = -1
            download_time_a = -1
        else:
            is_out_border = (cm[:,end_time] > t - period) & (cm[:,start_time] < t)
            if flag and len(cm[is_out_border]) != 0:
                last_chunk=cm[is_out_border]
                flag = False
            total_number_of_chunks_v = cm[(cm[:,type] == 0) & is_out_border].shape[0]
            if total_number_of_chunks_v == 0:
                avg_chunk_size_v = 0
            else:
                avg_chunk_size_v = cm[:,chunk_size][(cm[:,type] == 0) & (cm[:,end_time] > t - period) & (cm[:,start_time] < t)].mean()
            download_time_v = cm[:,download_time][(cm[:,type] == 0) & is_out_border].sum()


            total_number_of_chunks_a = cm[(cm[:,type] == 1) & is_out_border].shape[0]
            if total_number_of_chunks_a == 0:
                avg_chunk_size_a = 0
            else:
                avg_chunk_size_a = cm[:,chunk_size][(cm[:,type] == 1) & (cm[:,end_time] > t - period) & (cm[:,start_time] < t)].mean()
            download_time_a = cm[:,download_time][(cm[:,type] == 1) & is_out_border].sum()
        s += [total_number_of_chunks_v, avg_chunk_size_v, download_time_v,
              total_number_of_chunks_a, avg_chunk_size_a, download_time_a]
    # TODO
    # s += [离t最近的chunk，chunk的feature]

    s += list(last_chunk[-1])
    # s += list(last_chunk[0][-1])
    return s



In [6]:
filename = 'RequetDataSetNew/A0/PCAP_FILES/baseline_Jan17_exp_31.pcap'
# 1. chunkDetection
chunkNum, chunksValue0,fe_time = ChunkDetection(filename)
# 2. chunkMetrics
output = getChunkMetrics(chunkNum, chunksValue0)

getFeature(1516209864093,output,fe_time)

Reading pcap file...
Parsing pcap file...
Calcilate chunk...
56


[2,
 1279861.0,
 2.74914288520813,
 0,
 0,
 0.0,
 2,
 1279861.0,
 2.74914288520813,
 0,
 0,
 0.0,
 2,
 1279861.0,
 2.74914288520813,
 0,
 0,
 0.0,
 3,
 1492881.6666666667,
 4.736539840698242,
 0,
 0,
 0.0,
 3,
 1492881.6666666667,
 4.736539840698242,
 0,
 0,
 0.0,
 5,
 1412717.6,
 8.965434789657593,
 0,
 0,
 0.0,
 5,
 1412717.6,
 8.965434789657593,
 0,
 0,
 0.0,
 6,
 1524897.5,
 10.465993881225586,
 0,
 0,
 0.0,
 7,
 1564473.857142857,
 12.488471984863281,
 0,
 0,
 0.0,
 7,
 1564473.857142857,
 12.488471984863281,
 0,
 0,
 0.0,
 8,
 1445585.0,
 13.299987077713013,
 0,
 0,
 0.0,
 9,
 1495504.4444444445,
 15.083724021911621,
 0,
 0,
 0.0,
 9,
 1495504.4444444445,
 15.083724021911621,
 0,
 0,
 0.0,
 11,
 1420095.0,
 17.112349271774292,
 0,
 0,
 0.0,
 11,
 1420095.0,
 17.112349271774292,
 0,
 0,
 0.0,
 11,
 1420095.0,
 17.112349271774292,
 0,
 0,
 0.0,
 12,
 1464953.25,
 19.117595434188843,
 0,
 0,
 0.0,
 13,
 1400798.6923076923,
 19.849315404891968,
 0,
 0,
 0.0,
 14,
 1448151.642857143,


In [23]:
import pandas as pd
import csv
import glob
import os
ans = []
# for dataset in dataset_list:
dataset_folder = 'LabelDataSet/A0/'
files = glob.glob(dataset_folder + '*_label.csv')
# cnt=0
for i in range(1):
    file_name = files[i]
    print("Processing file: " + file_name)
    # if (i >= len_list[dataset]):
    # 	break
    file = pd.read_csv(file_name)  # wxh's csv name
    print(os.path.split(file_name)[1].split('/')[-1][:-17])
    feature_file = 'RequetDataSetNew/' +'A0'+'/PCAP_FILES/' + os.path.split(file_name)[1].split('/')[-1][:-17] + '.pcap'  # yhy's pcap name
    # 1. chunkDetection
    chunkNum, chunksValue0,fe_time = ChunkDetection(filename)
    # 2. chunkMetrics
    output = getChunkMetrics(chunkNum, chunksValue0)
    lable_size = file.shape[0]
    for i in range(0, lable_size, 50):
        feature = getFeature(file.iloc[i, 0],output,fe_time)
        if not feature:
            continue
        feature.append(file_name + "-" + str(file.iloc[i, 0]))
        feature.extend(file.iloc[i, 1:])
        ans.append(feature)
        print(feature[121])
# out_file = open('test_data/test_data111.csv', 'w', newline='')  # output   be careful of filename
# writer = csv.writer(out_file)
# keys = ['label' + str(i) for i in range(120)]
# keys.extend(['','','','','','',''])
# keys.extend(['filename_time', 'status', 'BuffWarning', 'Resolution'])
# writer.writerow(keys)
# for i in ans:
#     writer.writerow(i)
# out_file.close()

Processing file: LabelDataSet/A0\baseline_Jan17_exp_31_merged_label.csv
baseline_Jan17_exp_31
Reading pcap file...
Parsing pcap file...
Calcilate chunk...
56
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
1.0
1.0
1.0
1.0
1.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0


In [13]:
import pandas as pd
import csv
import glob
import os
ans = []
# for dataset in dataset_list:
dataset_folder = 'LabelDataSet/A0/'
files = glob.glob(dataset_folder + '*_label.csv')
# cnt=0
for i in range(1):
    file_name = files[i]
    print("Processing file: " + file_name)
    # if (i >= len_list[dataset]):
    # 	break
    file = pd.read_csv(file_name).to_numpy()  # wxh's csv name

    # print(os.path.split(file_name)[1].split('/')[-1][:-17])
    # feature_file = 'RequetDataSetNew/' +'A0'+'/PCAP_FILES/' + os.path.split(file_name)[1].split('/')[-1][:-17] + '.pcap'  # yhy's pcap name
    # # 1. chunkDetection
    # chunkNum, chunksValue0,fe_time = ChunkDetection(filename)
    # # 2. chunkMetrics
    # output = getChunkMetrics(chunkNum, chunksValue0)
    # lable_size = file.shape[0]
    # for i in range(0, lable_size, 50):
    #     feature = getFeature(file.iloc[i, 0],output,fe_time)
    #     if not feature:
    #         continue
    #     feature.append(file_name + "-" + str(file.iloc[i, 0]))
    #     feature.extend(file.iloc[i, 1:])
    #     ans.append(feature)
    #     print(feature[121])
# out_file = open('test_data/test_data111.csv', 'w', newline='')  # output   be careful of filename
# writer = csv.writer(out_file)
# keys = ['label' + str(i) for i in range(120)]
# keys.extend(['','','','','','',''])
# keys.extend(['filename_time', 'status', 'BuffWarning', 'Resolution'])
# writer.writerow(keys)
# for i in ans:
#     writer.writerow(i)
# out_file.close()

Processing file: LabelDataSet/A0\baseline_Jan17_exp_31_merged_label.csv


In [15]:
file.shape

(5768, 4)

In [10]:
servers = {sortChunks[0].getServerIP()}
for c in sortChunks:
    sIP = c.getServerIP()
    print(c.getServerIP())
    servers.add(sIP)

In [9]:
import matplotlib.pyplot as plt
fristTime = sortChunks[0].getGetTimestamp()
lastTime = sortChunks[len(sortChunks)-1].getDownEnd()

timeList1 = []
timeList2 = []
for c in sortChunks:
    if (c.getServerIP() == '173.194.7.39'):
        timeList1.append((c.getGetTimestamp()-fristTime, c.getDownEnd()-c.getGetTimestamp()))
    if (c.getServerIP() == '23.206.171.9'):
        timeList2.append((c.getGetTimestamp()-fristTime, c.getDownEnd()-c.getGetTimestamp()))

print(timeList2)
plt.figure(1,figsize=(20,4))
plt.broken_barh(timeList1, (0,1), facecolors ='cyan')
plt.broken_barh(timeList2, (2,1), facecolors ='green')
plt.xlim(0, 500)
#plt.xticks(np.arange(0, 500, 20))

NameError: name 'sortChunks' is not defined